<a href="https://colab.research.google.com/github/ardra1111/Neural-Networks/blob/main/Visualize_Adagrad_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt update
!sudo apt install libcairo2-dev ffmpeg \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install manim
!pip install IPython==8.21.0

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [872 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,395 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:13 http://security.ubuntu.com

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.0/810.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.8 MB/s eta 0:00:00
  Attempting uninstall: IPython
    Found existing installation: ipython 7.34.0
    Uninstalling ipython-7.34.0:
      Successfully uninstalled ipython-7.34.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 8.21.0 which is incompatible.


In [1]:
from manim import *

In [28]:
%%manim -ql AdaGrad
config.media_embed = True
from manim import *

class AdaGrad(Scene):
    def construct(self):
        self.camera.background_color = BLACK
        self.intro()
        self.show_explanation_and_formula()
        self.show_function()
        self.adagrad_steps()

    def intro(self):
        intro_text = Text("AdaGrad Algorithm", font_size=58, color=YELLOW)
        description = Text("An adaptive learning rate algorithm for gradient descent.", font_size=29)
        description.next_to(intro_text, DOWN)
        self.play(Write(intro_text), run_time=2)
        self.play(FadeIn(description), run_time=2)
        self.wait(2)
        self.play(FadeOut(intro_text), FadeOut(description))

    def show_explanation_and_formula(self):
        explanation = Text(
            "AdaGrad uses different learning rates for each parameter at each iteration.",
            font_size=27,
            color=WHITE
        ).to_edge(UP, buff=1.2)
        important_point = Text(
            "This is the key feature of AdaGrad!",
            font_size=28,
            color=WHITE
        ).next_to(explanation, DOWN, buff= 0.5)

        formula_title = Text("FORMULA", font_size=36, color=YELLOW).next_to(important_point, DOWN*2, buff= 0.625)
        formula = MathTex(
            r"\mathbf{W}_{t+1} = \mathbf{W}_t - \frac{\eta}{\sqrt{G_t + \epsilon}} \odot g_t",
            font_size=48,
            color=WHITE
        ).next_to(formula_title, DOWN,buff=0.9)

        self.play(Write(explanation), run_time=2)
        self.play(Write(important_point), run_time=2)
        self.wait(1)
        self.play(Write(formula_title), run_time=2)
        self.play(Write(formula), run_time=2)
        self.wait(2)
        self.play(FadeOut(explanation), FadeOut(important_point), FadeOut(formula_title), FadeOut(formula))

    def show_function(self):
        self.axes = Axes(
            x_range=[-3, 3, 1],
            y_range=[-1, 5, 1],
            axis_config={"color": BLUE},
        )
        graph = self.axes.plot(lambda x: x**2, color=GREEN)
        graph_label = self.axes.get_graph_label(graph, label="f(x) = x^2", x_val=2, direction=UR)

        self.play(Create(self.axes), run_time=2)
        self.play(Create(graph), run_time=2)
        self.play(FadeIn(graph_label))
        self.wait(2)

    def adagrad_steps(self):
        start_point = np.array([-2, 4, 0])
        steps = [start_point]
        learning_rate = 1.0
        epsilon = 1e-8
        gradient_accumulated = 0

        # Generating steps
        for _ in range(10):
            grad = 2 * steps[-1][0]  # Gradient of f(x) = x^2
            gradient_accumulated += grad**2
            adjusted_lr = learning_rate / (np.sqrt(gradient_accumulated) + epsilon)
            next_point = steps[-1][0] - adjusted_lr * grad
            steps.append(np.array([next_point, next_point**2, 0]))

        points = VGroup(*[Dot(self.axes.c2p(*pt[:2]), color=RED) for pt in steps])
        lines = VGroup(*[Line(self.axes.c2p(*steps[i][:2]), self.axes.c2p(*steps[i+1][:2]), color=YELLOW) for i in range(len(steps)-1)])

        self.play(Create(points[0]))

        epoch_text = Text("Epoch: 0", font_size=24)
        epoch_text.to_edge(LEFT).shift(DOWN*1.5)
        self.play(FadeIn(epoch_text))

        for i in range(len(steps)-1):
            self.play(Create(lines[i]))
            self.play(Create(points[i+1]))

            grad = 2 * steps[i][0]
            gradient_accumulated += grad**2
            adjusted_lr = learning_rate / (np.sqrt(gradient_accumulated) + epsilon)
            lr_text = Text(f"Learning Rate: {adjusted_lr:.4f}", font_size=24, color=WHITE)
            lr_text.next_to(self.axes.c2p(*steps[i+1][:2]), UP)

            # Update epoch text
            self.play(Transform(epoch_text, Text(f"Epoch: {i+1}", font_size=24).to_edge(LEFT).shift(DOWN*1.5)))

            self.play(GrowArrow(Arrow(self.axes.c2p(*steps[i][:2]), self.axes.c2p(*steps[i+1][:2]), buff=0, color =PINK)))
            self.play(FadeIn(lr_text))
            self.wait(1)
            self.play(FadeOut(lr_text))

        # Ensure the final epoch number remains
        self.play(FadeOut(epoch_text))
        final_epoch_text = Text(f"Final Epoch: {len(steps)-1}", font_size=24)
        final_epoch_text.to_edge(LEFT).shift(DOWN*1.5)
        self.play(FadeIn(final_epoch_text))

        self.wait(3)

if __name__ == "__main__":
    from manim import *
    config.pixel_height = 720
    config.pixel_width = 1280
    config.frame_rate = 30
    scene = AdaGrad11()
    scene.render()


[05/23/24 19:48:22] INFO     Animation 0 : Using cached data (hash :                           ]8;id=189208;file:///usr/local/lib/python3.10/dist-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=652165;file:///usr/local/lib/python3.10/dist-packages/manim/renderer/cairo_renderer.py#88\88]8;;\
                             2016333726_3402263429_223132457)                                                      

INFO:manim:Animation 0 : Using cached data (hash : 2016333726_3402263429_223132457)


[05/23/24 19:48:25] INFO     Animation 1 : Partial movie file written in                   ]8;id=383935;file:///usr/local/lib/python3.10/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=375434;file:///usr/local/lib/python3.10/dist-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/content/media/videos/720p30/partial_movie_files/AdaGrad11/5                         
                             43634251_819169299_204298192.mp4'                                                     

INFO:manim:Animation 1 : Partial movie file written in '/content/media/videos/720p30/partial_movie_files/AdaGrad11/543634251_819169299_204298192.mp4'


[05/23/24 19:48:26] INFO     Animation 2 : Using cached data (hash :                           ]8;id=432886;file:///usr/local/lib/python3.10/dist-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=336452;file:///usr/local/lib/python3.10/dist-packages/manim/renderer/cairo_renderer.py#88\88]8;;\
                             543634251_784179552_3976409467)                                                       

...

INFO:manim:
File ready at '/content/media/videos/content/480p15/AdaGrad.mp4'



                    INFO     The partial movie directory is full (> 100 files). Therefore, ]8;id=342772;file:///usr/local/lib/python3.10/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=168811;file:///usr/local/lib/python3.10/dist-packages/manim/scene/scene_file_writer.py#707\707]8;;\
                             manim has removed the 74 oldest file(s). You can change this                          
                             behaviour by changing max_files_cached in config.                                     

INFO:manim:The partial movie directory is full (> 100 files). Therefore, manim has removed the 74 oldest file(s). You can change this behaviour by changing max_files_cached in config.


                    INFO     Rendered AdaGrad                                                          ]8;id=744624;file:///usr/local/lib/python3.10/dist-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=862671;file:///usr/local/lib/python3.10/dist-packages/manim/scene/scene.py#247\247]8;;\
                             Played 90 animations                                                                  

INFO:manim:Rendered AdaGrad
Played 90 animations
